In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import letRun #This library can be deleted, it is used for debugging
import RoboSkin as sk
import cv2 
import pickle
from sklearn import preprocessing
import os

#if linux
import matplotlib
matplotlib.use('TkAgg')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.version.cuda)
print("GPU:",torch.cuda.is_available())

path="C:/Users/dexte/github/RoboSkin/Code/Models/labeller/"
if os.name == 'nt':
    path="C:/Users/dexte/github/RoboSkin/Code/Models/labeller/" #use standard imputer or one for small
else:
    path="/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/labeller/" #for linux
    
def sigmoid(x):                                        
   return 1 / (1 + np.exp(-x))

11.7
GPU: True


In [2]:
##DATA SET CREATOR
class dataset:
    def __init__(self,names=["flat_detection.avi","edge_detection.avi","flat_slip_detection.avi","soft_detection.avi","flat_slip_detection_paper.avi","nothing.avi"]):
        self.path=letRun.path
        self.names=names
        self.SIZE=0.3
        name=""
        if os.name == 'nt':
            name="C:/Users/dexte/OneDrive/Documents/AI/Data_Labeller/pickle_imputer_small.pkl" #use standard imputer or one for small
        else:
            name="/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/TacTip reader/pickle_imputer_small.pkl" #for linux
        self.reg=None
        with open(name,'rb') as file:
            self.reg=pickle.load(file)
    def predict(self,reg,dat):
        p=reg.predict(dat)
        p=(p.reshape((p.shape[0],p.shape[1]//2,2))*255/self.SIZE)
        return p
    def generate(self,STORE=5,y_labels=[],scale=False,gyro_=True):
        BIG_DATA_X=None
        BIG_DATA_y=None
        assert len(y_labels)>=len(self.names),"Incorrect length of labels"
        for j,name in enumerate(self.names):
            skin=sk.Skin(videoFile=self.path+name)#videoFile=path+"Movement4.avi") #load skin object using demo video
            cap = cv2.VideoCapture(self.path+name)
            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            skin.sharpen=False #falsify the sharpness if recorded with sharpness
            frame=skin.getFrame()
            h=frame.shape[1]*self.SIZE
            w=frame.shape[0]*self.SIZE
            frame=cv2.resize(frame,(int(h),int(w)),interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).flatten()/255
            past=self.predict(self.reg,np.array([frame]))[0]
            initial=past.copy()
            initial_frame=frame.copy()
            counter=0
            X=[]
            y=[] #label as [edge surface soft hard slippery]
            lastFrames=[]
            gyro=np.zeros((length,6))
            try:
                gyro=np.load(self.path+name.replace(".avi","")+"_gyro.npy")
            except:
                pass
            for i in range(length): #lop through all
                frame_=skin.getFrame()
                frame=cv2.resize(frame_,(int(h),int(w)),interpolation=cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).flatten()/255
                points=self.predict(self.reg,np.array([frame]))[0]
                #get pressure map
                diff=np.sum(np.abs(initial_frame-frame))/(frame.shape[0]*3)
                vecs=initial-points
                if gyro_:
                    lastFrames.append(np.concatenate((vecs.flatten(),gyro[i]*1)))
                else:
                    lastFrames.append(vecs.flatten())
                if len(lastFrames)>STORE: lastFrames.pop(0)
                if diff>0.01 and len(lastFrames)==STORE: #significant contact
                    X.append(np.array(lastFrames).flatten()) #store temporal element
                    y.append(y_labels[j])
                    counter+=1

            if type(BIG_DATA_y)==type(None):
                BIG_DATA_y=np.array(y.copy())
                BIG_DATA_X=np.array(X.copy())
            else:
                BIG_DATA_y= np.concatenate((np.array(y.copy()),BIG_DATA_y))
                BIG_DATA_X= np.concatenate((np.array(X.copy()),BIG_DATA_X))
            print(name,"Length:",counter)
        
        a,b=np.array(BIG_DATA_X),np.array(BIG_DATA_y)
        if scale:
            """scaler1 = preprocessing.StandardScaler().fit(a)
            a = scaler1.transform(a)"""
            a=(a-np.average(a))/np.std(a)
        return a,b
    def gen_image_data(self,STORE=5,y_labels=[],scale=False):
        BIG_DATA_X=None
        BIG_DATA_y=None
        assert len(y_labels)>=len(self.names),"Incorrect length of labels"
        for j,name in enumerate(self.names):
            skin=sk.Skin(videoFile=self.path+name)#videoFile=path+"Movement4.avi") #load skin object using demo video
            cap = cv2.VideoCapture(self.path+name)
            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            skin.sharpen=False #falsify the sharpness if recorded with sharpness
            frame=skin.getFrame()
            h=frame.shape[1]*self.SIZE
            w=frame.shape[0]*self.SIZE
            frame=cv2.resize(frame,(int(h),int(w)),interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).flatten()/255
            
            initial_frame=frame.copy()
            counter=0
            X=[]
            y=[] #label as [edge surface soft hard slippery]
            lastFrames=[]
            gyro=np.zeros((length,6))
            try:
                gyro=np.load(self.path+name.replace(".avi","")+"_gyro.npy")
            except:
                pass
            for i in range(length): #lop through all
                frame_=skin.getFrame()
                frame=cv2.resize(frame_,(int(h),int(w)),interpolation=cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).flatten()/255
                #get pressure map
                diff=np.sum(np.abs(initial_frame-frame))/(frame.shape[0]*3)
                
                lastFrames.append(np.concatenate((frame.flatten(),gyro[i]*1)))
                if len(lastFrames)>STORE: lastFrames.pop(0)
                if diff>0.01 and len(lastFrames)==STORE: #significant contact
                    X.append(np.array(lastFrames).flatten()) #store temporal element
                    y.append(y_labels[j])
                    counter+=1

            if type(BIG_DATA_y)==type(None):
                BIG_DATA_y=np.array(y.copy())
                BIG_DATA_X=np.array(X.copy())
            else:
                BIG_DATA_y= np.concatenate((np.array(y.copy()),BIG_DATA_y))
                BIG_DATA_X= np.concatenate((np.array(X.copy()),BIG_DATA_X))
            print(name,"Length:",counter,"X-size",BIG_DATA_X[0].shape)
        print("Completed creation")
        if scale:
            """scaler1 = preprocessing.StandardScaler().fit(a)
            a = scaler1.transform(a)"""
            BIG_DATA_X=(BIG_DATA_X-np.average(BIG_DATA_X))/np.std(BIG_DATA_X)
        return BIG_DATA_X,BIG_DATA_y
    def generate_average(self,STORE=5,y_labels=[],scale=False):
        BIG_DATA_X=None
        BIG_DATA_y=None
        assert len(y_labels)>=len(self.names),"Incorrect length of labels"
        for j,name in enumerate(self.names):
            skin=sk.Skin(videoFile=self.path+name)#videoFile=path+"Movement4.avi") #load skin object using demo video
            cap = cv2.VideoCapture(self.path+name)
            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            skin.sharpen=False #falsify the sharpness if recorded with sharpness
            frame=skin.getFrame()
            h=frame.shape[1]*self.SIZE
            w=frame.shape[0]*self.SIZE
            frame=cv2.resize(frame,(int(h),int(w)),interpolation=cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).flatten()/255
            past=self.predict(self.reg,np.array([frame]))[0]
            initial=past.copy()
            initial_frame=frame.copy()

            X=[]
            y=[] #label as [edge surface soft hard slippery]
            lastFrames=[]
            gyro=np.zeros((length,6))
            try:
                gyro=np.load(self.path+name.replace(".avi","")+"_gyro.npy")
            except:
                pass
            counter=0
            for i in range(length): #lop through all
                frame_=skin.getFrame()
                frame=cv2.resize(frame_,(int(h),int(w)),interpolation=cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).flatten()/255
                points=self.predict(self.reg,np.array([frame]))[0]
                #get pressure map
                diff=np.sum(np.abs(initial_frame-frame))/(frame.shape[0]*3)
                vecs=initial-points
                average=np.average(vecs,axis=0).flatten()
                lastFrames.append(np.concatenate((average,gyro[i]/1)))
                if len(lastFrames)>STORE: lastFrames.pop(0)
                if diff>0.01 and len(lastFrames)==STORE: #significant contact
                    X.append(np.array(lastFrames).flatten()) #store temporal element
                    y.append(y_labels[j])
                    counter+=1
            if type(BIG_DATA_y)==type(None):
                BIG_DATA_y=np.array(y.copy())
                BIG_DATA_X=np.array(X.copy())
            else:
                BIG_DATA_y= np.concatenate((np.array(y.copy()),BIG_DATA_y))
                BIG_DATA_X= np.concatenate((np.array(X.copy()),BIG_DATA_X))
            print(name,"Length:",counter)
        a,b=np.array(BIG_DATA_X),np.array(BIG_DATA_y)
        #a=a.reshape((a.shape))
        #regulise
        if scale:
            scaler1 = preprocessing.StandardScaler().fit(a)
            a = scaler1.transform(a)
        return a,b


### Main code

In [22]:

d=dataset()
d.SIZE=0.3
X,ya=d.generate(STORE=5,y_labels=[[1,0,0,0],[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],scale=True)
#x,y =d.generate(STORE=5,y_labels=[[1,0,0],[1,0,0],[0,0,1]],scale=True)
print(np.sum(X))
#Xa=X/10
ya=ya/10
#y=y/10
X, data_test, Y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
#X, data_test, Y, labels_test = train_test_split(x, y , test_size=0.20, random_state=42)

print(X.shape,Y.shape)
print(data_test.shape,labels_test.shape)

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 683
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
soft_detection.avi Length: 238
flat_slip_detection_paper.avi Length: 630
nothing.avi Length: 510
-1.90084392670542e-10
(2120, 1360) (2120, 4)
(530, 1360) (530, 4)


In [4]:


# Define the neural network model
class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size,layers=[5000,500,50],drop_out_prob=0.2):
        super(SimpleNeuralNetwork, self).__init__()
        self.fc=[nn.Linear(input_size, layers[0])]
        self.fc.append(nn.Sigmoid())
        self.fc.append(nn.Dropout(p=drop_out_prob))
        for i in range(1,len(layers)): #create layers 
                self.fc.append(nn.Linear(layers[i-1], layers[i]))
                self.fc.append(nn.Sigmoid())
                self.fc.append(nn.Dropout(p=drop_out_prob))
        self.fc.append(nn.Linear(layers[-1], output_size))
        self.fc_layers = nn.Sequential(*self.fc)
    def forward(self, x):
        x=self.fc_layers(x)
        return x





In [ ]:

# Define the size of the input (n) and output (m) layers
n_inputs = X.shape[1]
m_outputs = len(Y[0])


# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)

# Create the neural network
model = SimpleNeuralNetwork(n_inputs, m_outputs,drop_out_prob=0.1).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()#nn.MSELoss() #nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.02)

In [5]:
# Training loop
def get_acc(predictions,should_be=[]):
    predictions*=10 #normalize
    pred_array=np.zeros_like(predictions)

    if len(predictions[0])>4:
        inds=np.argmax(predictions[:,0:2],axis=1) #convert at threshold
        inds2=np.argmax(predictions[:,2:-1],axis=1) #convert at threshold
        for i in range(len(pred_array)):
            pred_array[i][inds[i]]=1
            pred_array[i][2+inds2[i]]=1
    else:
        inds=np.argmax(predictions,axis=1) #convert at threshold
        for i in range(len(pred_array)):
            pred_array[i][inds[i]]=1
    correct=0
    for j,pred in enumerate(pred_array): #loopthrough predictions
        inds=np.where(pred==1) #
        if len(should_be)>0: #validation task
            sb=should_be[j]*10
            sb=np.where(sb==1)
            if len(sb[0])==len(inds[0]):
                c=0
                for i in range(len(sb[0])):
                    if sb[0][i]==inds[0][i]: c+=1
                if c==len(sb[0]): correct+=1
    return correct/len(should_be)

def train(model,num_epochs=2500,output=True):
    loss_ar=[]
    accuracies=[]
    for epoch in range(num_epochs):
        # Forward pass
        y_pred = model(X_tensor)

        # Compute the loss
        loss = criterion(y_pred, y_tensor)

        # Zero gradients, backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_ar.append(loss.item())
        #predict
        predictions = model(torch.tensor(X, dtype=torch.float32).to(device))
        accuracies.append(get_acc(predictions.cpu().detach().numpy(),should_be=Y))
        # Print the current loss to monitor training progress
        if epoch%1000==0 and output:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}","Accuracy",accuracies[-1])
    return np.array(loss_ar),np.array(accuracies)



In [20]:
plt.title("Loss while training tactile model")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
a,b=train(model,150000)
#plt.plot(a,label="Loss")
plt.plot(b,label="Accuracy")
plt.legend(loc="lower right")
plt.show()

Epoch [1/150000], Loss: 0.1364 Accuracy 0.165


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/surfaceModel/state/model1.pth")

In [ ]:
# After training, you can use the trained model for predictions on new data.
# For example, if you have new input data 'X_new', you can do:
with torch.no_grad():
     model.eval()
     predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))


def predict(predictions,should_be=[]):
    predictions*=10 #normalize
    pred_array=np.zeros_like(predictions)
    if len(predictions[0])>4:
        inds=np.argmax(predictions[:,0:2],axis=1) #convert at threshold
        inds2=np.argmax(predictions[:,2:-1],axis=1) #convert at threshold
        for i in range(len(pred_array)):
            pred_array[i][inds[i]]=1
            pred_array[i][2+inds2[i]]=1
    else:
        inds=np.argmax(predictions,axis=1) #convert at threshold
        for i in range(len(pred_array)):
            pred_array[i][inds[i]]=1
    names=["edge", "surface", "soft", "hard", "slippery"]
    array=[]
    correct=0
    for j,pred in enumerate(pred_array): #loopthrough predictions
        inds=np.where(pred==1) #
        ar=[]
        if len(should_be)>0: #validation task
            sb=should_be[j]*10
            sb=np.where(sb==1)
            if len(sb[0])==len(inds[0]):
                c=0
                for i in range(len(sb[0])):
                    if sb[0][i]==inds[0][i]: c+=1
                if c==len(sb[0]): correct+=1
        for i in inds[0]:
            ar.append(names[i])
        array.append(ar)
    if correct!=0: print("Percentage correct:",(correct/len(should_be))*100,"%")
    return array

p=predict(predictions.cpu(),should_be=labels_test)
p[0:20]  

Percentage correct: 96.79245283018868 %


[['soft'],
 ['surface'],
 ['soft'],
 ['edge'],
 ['soft'],
 ['soft'],
 ['edge'],
 ['soft'],
 ['soft'],
 ['edge'],
 ['edge'],
 ['edge'],
 ['soft'],
 ['hard'],
 ['hard'],
 ['edge'],
 ['hard'],
 ['surface'],
 ['hard'],
 ['edge']]

# Experiments

### With and without slippage

In [ ]:
TRIALS=100
#without slippage
#merge datasets
d=dataset(names=["flat_detection.avi","edge_detection.avi"])
X,ya=d.generate(STORE=5,y_labels=[[0,1,0,1,0],[1,0,0,1,0]])
Xa=X/100
ya=ya/10

X, data_test, y, labels_test = train_test_split(Xa, ya, test_size=0.20, random_state=42)

average_a=np.zeros((2500,))
p_=0

for i in range(TRIALS):
    print("Trial B",i,"Acc:",p_/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    l,a=train(model)
    average_a+=np.array(a)
    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p_+=get_acc(predictions.cpu(),should_be=labels_test)

average_a=average_a/(TRIALS) #average by 10 and then divide by 5 for scaling
p_/=TRIALS

average_a1=np.zeros((2500,))

d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[0,1,0,1,0],[1,0,0,1,0],[0,1,0,0,1]])
Xa=X/100
ya=ya/10

X, data_test, y, labels_test = train_test_split(Xa, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

p=0
for i in range(TRIALS):
    print("Trial A",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    l,a=train(model)
    average_a1+=np.array(a)

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p+=get_acc(predictions.cpu(),should_be=labels_test)
average_a1=average_a1/(TRIALS) #average by 10 and then divide by 5 for scaling
p/=TRIALS
#plt.plot(average_l,'--',c="b",label="Loss with slip data")
plt.plot(average_a1,c="b",label="Accuracy with slip data")


print("Accuracies",p,p_)
plt.title("Performance of models with and without slippage over 100 trials")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.plot(average_l,c="r",label="Loss without slip data")
plt.plot(average_a,c="r",label="Accuracy without slip data")
plt.legend(loc="lower right")
plt.show()

print(np.std(average_a),np.std(average_a1))

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
np.save(path+"BignetNoSlip",average_a)
np.save(path+"Bignet",average_a1)


ar1=np.load(path+"40netNoSlip.npy")
ar2=np.load(path+"40net.npy")
plt.title("Performance of models with and without slippage over 100 trials")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.plot(average_l,c="r",label="Loss without slip data")
plt.plot(ar1,c="g",label="Accuracy compressed input without slip data")
plt.plot(ar2,c="y",label="Accuracy compressed input")
plt.plot(average_a,c="r",label="Accuracy without slip data")
plt.plot(average_a1,c="b",label="Accuracy")
plt.legend(loc="lower right")
plt.show()


### With and without gyroscopic data


In [20]:
num_epochs=120000
gyro=np.zeros((num_epochs,))
non_gyro=np.zeros((num_epochs,))
TRIALS=15
d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[1,0,0,0],[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],scale=True)
ya=ya/10

X, data_test, Y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
n_inputs=X.shape[1]
m_outputs=Y.shape[1]
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)


for i in range(TRIALS):
    model = SimpleNeuralNetwork(n_inputs, m_outputs,layers=[100,50,10]).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs,output=False)
    

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    gyro+=a#get_acc(predictions.cpu(),should_be=labels_test)
    print("A",np.max(gyro)/(i+1))

d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[1,0,0,0],[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],scale=True,gyro_=False)
n_inputs=X.shape[1]
ya=ya/10

X, data_test, Y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)


for i in range(TRIALS):
    model = SimpleNeuralNetwork(n_inputs, m_outputs,layers=[100,50,10]).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs,output=False)
    

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    non_gyro+=a#get_acc(predictions.cpu(),should_be=labels_test)
    print("B",non_gyro[-1]/(i+1))


non_gyro/=TRIALS
gyro/=TRIALS

plt.title("Accuracy training tactile model")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

#plt.plot(a,label="Loss")
plt.plot(gyro,label="Accuracy of gyro")
plt.plot(non_gyro,label="Accuracy of non-gyro")
plt.legend(loc="lower right")
plt.show()

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 683
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
soft_detection.avi Length: 238
flat_slip_detection_paper.avi Length: 630
nothing.avi Length: 510
A 0.39716981132075474
A 0.39528301886792455
A 0.3845911949685535
A 0.3824292452830188
A 0.38122641509433963
A 0.3810534591194969
A 0.3789757412398922
A 0.37841981132075475
A 0.37777777777777777
A 0.3774056603773585
A 0.37838765008576336
A 0.37716194968553457
A 0.37681422351233673
A 0.37665094339622635
A 0.3759119496855346


/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 683
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
soft_detection.avi Length: 238
flat_slip_detection_paper.avi Length: 630
nothing.avi Length: 510
B 0.3650943396226415
B 0.36721698113207546
B 0.37028301886792453
B 0.37016509433962264
B 0.3692452830188679
B 0.370440251572327
B 0.37028301886792453
B 0.3722877358490566
B 0.37180293501048217
B 0.3715566037735849
B 0.37259862778730707
B 0.37358490566037733
B 0.3740203193033382


In [18]:
np.max(non_gyro),np.max(gyro)

(0.3490566037735849, 0.35056603773584905)

In [14]:
plt.title("Accuracy training tactile model")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

#plt.plot(a,label="Loss")
plt.plot(gyro,label="Accuracy of gyro")
plt.plot(non_gyro,label="Accuracy of non-gyro")
plt.legend(loc="lower right")
plt.show()

### With or without compression

In [ ]:
TRIALS=50
num_epochs=3500
average_a1=np.zeros((num_epochs,))

d=dataset()
X,ya=d.generate_average(STORE=5,y_labels=[[0,1,0],[0,1,0],[0,0,1]],scale=True)
ya=ya/10
n_inputs=X.shape[1]
X, data_test, y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
m_outputs=3
p=0
for i in range(TRIALS):
    print("Trial A",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs,layers=[100,50,10]).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs)
    average_a1+=np.array(a)

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p+=get_acc(predictions.cpu(),should_be=labels_test)
average_a1=average_a1/(TRIALS) #average by 10 and then divide by 5 for scaling
p/=TRIALS
#plt.plot(average_l,'--',c="b",label="Loss with slip data")

average_a2=np.zeros((num_epochs,))

d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[0,1,0],[0,1,0],[0,0,1]],scale=True)
ya=ya/10
n_inputs=X.shape[1]
X, data_test, y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
m_outputs=3
p_=0
for i in range(TRIALS):
    print("Trial A",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs)
    average_a2+=np.array(a)

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p_+=get_acc(predictions.cpu(),should_be=labels_test)
average_a2=average_a2/(TRIALS) #average by 10 and then divide by 5 for scaling
p_/=TRIALS
#plt.plot(average_l,'--',c="b",label="Loss with slip data")
plt.plot(average_a1,c="b",label="With compression")


plt.title("Performance of models with and without slippage over 50 trials")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.plot(average_l,c="r",label="Loss without slip data")
plt.plot(average_a2,c="r",label="Without compression")
plt.legend(loc="lower right")
plt.show()

print(np.std(average_a),np.std(average_a1))

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
Trial A 0 Acc: 0.7710752688172042
Epoch [1/3500], Loss: 0.0696 Accuracy 0.3468286099865047
Epoch [1001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [2001/3500], Loss: 0.0000 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0000 Accuracy 1.0
Trial A 1 Acc: 0.3855376344086021
Epoch [1/3500], Loss: 0.0282 Accuracy 0.029689608636977057
Epoch [1001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [2001/3500], Loss: 0.0000 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0000 Accuracy 1.0
Trial A 2 Acc: 0.2570250896057347
Epoch [1/3500], Loss: 0.0795 Accuracy 0.3427800269905533
Epoch [1001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [2001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0000 Accuracy 1.0
Trial A 3 Acc: 0.19276881720430106
Epoch [1/3500], Loss: 0.1012 Accuracy 0.4008097165991903
Epoch [1001/3500], Loss: 0.0001 Accuracy 0.9973009446693657
Epoch [2001/3500], Loss: 0.0001 Accuracy 0.9986504723346828
Epoch [300

In [ ]:
plt.title("Performance of models with and without slippage over 50 trials")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.plot(average_a1,c="b",label="With compression")
plt.plot(average_a2,c="r",label="Without compression")
plt.legend(loc="lower right")
plt.show()

### With or without preprocessing

In [ ]:
TRIALS=50
num_epochs=3500
d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[0,1,0,1,0],[1,0,0,1,0],[0,1,0,0,1]],scale=True)
ya=ya/10
n_inputs=X.shape[1]
X, data_test, y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
average_a1=np.zeros((num_epochs,))
p=0
for i in range(TRIALS):
    print("Trial A",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs)
    average_a1+=np.array(a)

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p+=get_acc(predictions.cpu(),should_be=labels_test)
average_a1=average_a1/(TRIALS) #average by 10 and then divide by 5 for scaling

d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[0,1,0,1,0],[1,0,0,1,0],[0,1,0,0,1]],scale=False)
X=X/100
ya=ya/10

X, data_test, y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
average_a=np.zeros((num_epochs,))
p=0
for i in range(TRIALS):
    print("Trial B",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs)
    average_a+=np.array(a)

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p+=get_acc(predictions.cpu(),should_be=labels_test)
average_a=average_a/(TRIALS) #average by 10 and then divide by 5 for scaling

plt.title("Performance of models with and without preprocessing")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.plot(average_l,c="r",label="Loss without slip data")
plt.plot(average_a,c="r",label="Without preprocessing")
plt.plot(average_a1,c="b",label="With preprocessing")
plt.legend(loc="lower right")
plt.show()

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
Trial A 0 Acc: 0.0
Epoch [1/3500], Loss: 0.1813 Accuracy 0.3643724696356275
Epoch [1001/3500], Loss: 0.0120 Accuracy 0.22267206477732793
Epoch [2001/3500], Loss: 0.0077 Accuracy 0.24561403508771928
Epoch [3001/3500], Loss: 0.0055 Accuracy 0.26450742240215924
Trial A 1 Acc: 0.17204301075268819
Epoch [1/3500], Loss: 0.0760 Accuracy 0.3252361673414305
Epoch [1001/3500], Loss: 0.0130 Accuracy 0.19973009446693657
Epoch [2001/3500], Loss: 0.0085 Accuracy 0.242914979757085
Epoch [3001/3500], Loss: 0.0059 Accuracy 0.28475033738191635
Trial A 2 Acc: 0.22939068100358426
Epoch [1/3500], Loss: 0.1676 Accuracy 0.17813765182186234
Epoch [1001/3500], Loss: 0.0132 Accuracy 0.2321187584345479
Epoch [2001/3500], Loss: 0.0084 Accuracy 0.22941970310391363
Epoch [3001/3500], Loss: 0.0059 Accuracy 0.24561403508771928
Trial A 3 Acc: 0.25806451612903225
Epoch [1/3500], Loss: 0.0617 Accuracy 0.17408906882591094
Ep

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
Trial B 0 Acc: 0.0
Epoch [1/3500], Loss: 0.0708 Accuracy 0.006747638326585695
Epoch [1001/3500], Loss: 0.0149 Accuracy 0.21997300944669365
Epoch [2001/3500], Loss: 0.0092 Accuracy 0.22267206477732793
Epoch [3001/3500], Loss: 0.0065 Accuracy 0.26180836707152494
Trial B 1 Acc: 0.17204301075268819
Epoch [1/3500], Loss: 0.1317 Accuracy 0.19838056680161945
Epoch [1001/3500], Loss: 0.0133 Accuracy 0.19838056680161945
Epoch [2001/3500], Loss: 0.0088 Accuracy 0.24021592442645073
Epoch [3001/3500], Loss: 0.0062 Accuracy 0.2483130904183536
Trial B 2 Acc: 0.22939068100358426
Epoch [1/3500], Loss: 0.1021 Accuracy 0.3157894736842105
Epoch [1001/3500], Loss: 0.0142 Accuracy 0.2078272604588394
Epoch [2001/3500], Loss: 0.0091 Accuracy 0.24426450742240216
Epoch [3001/3500], Loss: 0.0060 Accuracy 0.2766531713900135
Trial B 3 Acc: 0.25806451612903225
Epoch [1/3500], Loss: 0.1501 Accuracy 0.3090418353576248
E

### Architecture

In [ ]:
TRIALS=50
num_epochs=3500
d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[1,0],[1,0],[0,1]],scale=True)
ya=ya/10
n_inputs=X.shape[1]
m_outputs=2
X, data_test, y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
average_a1=np.zeros((num_epochs,))
p=0
for i in range(TRIALS):
    print("Trial A",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs)
    average_a1+=np.array(a)

    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p+=get_acc(predictions.cpu(),should_be=labels_test)
average_a1=average_a1/(TRIALS) #average by 10 and then divide by 5 for scaling


d=dataset()
X,ya=d.generate(STORE=5,y_labels=[[0,1,0,1,0],[1,0,0,1,0],[0,1,0,0,1]],scale=True)
ya=ya/10
n_inputs=X.shape[1]
m_outputs=5
X, data_test, y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
average_a2=np.zeros((num_epochs,))
p=0
for i in range(TRIALS):
    print("Trial A",i,"Acc:",p/(i+1))
    # Create the neural network
    model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    l,a=train(model,num_epochs)
    average_a2+=np.array(a)
 
    #unseen data
    with torch.no_grad():
        model.eval()
        predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
    p+=get_acc(predictions.cpu(),should_be=labels_test)
average_a2=average_a2/(TRIALS) #average by 10 and then divide by 5 for scaling

plt.title("How the architecture influences model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.plot(average_l,c="r",label="Loss without slip data")
plt.plot(average_a1,c="g",label="binary-classifier")
plt.plot(average_a2,c="y",label="Multi-classifier")
plt.legend(loc="lower right")
plt.show()

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
Trial A 0 Acc: 0.0
Epoch [1/3500], Loss: 0.0741 Accuracy 0.46153846153846156
Epoch [1001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [2001/3500], Loss: 0.0000 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0000 Accuracy 1.0
Trial A 1 Acc: 0.49731182795698925
Epoch [1/3500], Loss: 0.1171 Accuracy 0.6572199730094467
Epoch [1001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [2001/3500], Loss: 0.0000 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0000 Accuracy 1.0
Trial A 2 Acc: 0.6630824372759857
Epoch [1/3500], Loss: 0.3727 Accuracy 0.6410256410256411
Epoch [1001/3500], Loss: 0.0002 Accuracy 0.9986504723346828
Epoch [2001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0001 Accuracy 1.0
Trial A 3 Acc: 0.7459677419354839
Epoch [1/3500], Loss: 0.1453 Accuracy 0.6572199730094467
Epoch [1001/3500], Loss: 0.0001 Accuracy 1.0
Epoch [2001/3500], Loss: 0.0000 Accuracy 1.0
Epoch [3001/3500], Loss: 0.0000 Accuracy 

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
Trial A 0 Acc: 0.0
Epoch [1/3500], Loss: 0.0606 Accuracy 0.18083670715249664
Epoch [1001/3500], Loss: 0.0001 Accuracy 0.6477732793522267
Epoch [2001/3500], Loss: 0.0001 Accuracy 0.6558704453441295
Epoch [3001/3500], Loss: 0.0001 Accuracy 0.6572199730094467
Trial A 1 Acc: 0.2768817204301075
Epoch [1/3500], Loss: 0.1013 Accuracy 0.36707152496626183
Epoch [1001/3500], Loss: 0.0002 Accuracy 0.6450742240215924
Epoch [2001/3500], Loss: 0.0001 Accuracy 0.6383265856950068
Epoch [3001/3500], Loss: 0.0001 Accuracy 0.6572199730094467
Trial A 2 Acc: 0.37275985663082434
Epoch [1/3500], Loss: 0.0805 Accuracy 0.28475033738191635
Epoch [1001/3500], Loss: 0.0001 Accuracy 0.6545209176788124
Epoch [2001/3500], Loss: 0.0001 Accuracy 0.6572199730094467
Epoch [3001/3500], Loss: 0.0001 Accuracy 0.6558704453441295
Trial A 3 Acc: 0.42607526881720426
Epoch [1/3500], Loss: 0.1133 Accuracy 0.10796221322537113
Epoch [

In [ ]:
plt.title("How the architecture influences model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
#plt.plot(average_l,c="r",label="Loss without slip data")
plt.plot(average_a1,c="g",label="binary-classifier")
plt.plot(average_a2,c="y",label="Multi-classifier")
plt.legend(loc="lower right")
plt.show()

In [ ]:
average_a2

array([0., 0., 0., ..., 0., 0., 0.])

### Store size

In [ ]:
TRIALS=20
TIMES=15
average_a=np.zeros((TIMES,800))
average_unseen=np.zeros((TIMES,))
for i in range(1,TIMES):
    d=dataset()
    X,ya=d.generate(STORE=i,y_labels=[[1,0,0],[1,0,0],[0,0,1]],scale=True)
    ya=ya/10
    X, data_test, Y, labels_test = train_test_split(X, ya, test_size=0.20, random_state=42)
    n_inputs = X.shape[1]
    m_outputs = len(y[0])
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)
    for j in range(TRIALS):
        model = SimpleNeuralNetwork(n_inputs, m_outputs).to(device)
        # Define the loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        l,a=train(model,800,output=False)
        #predict
        with torch.no_grad():
            model.eval()
            predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device))
        acc=get_acc(predictions.cpu(),should_be=labels_test)
        average_unseen[i]+=acc

        average_a[i]+=a
    average_a[i]/=TRIALS #get average
    average_unseen[i]/=TRIALS
    print(i,average_a[i][-1],average_unseen[i])
#diplay

plt.title("Accuricies vs T size")
plt.plot(np.max(average_a,axis=1))
plt.ylabel("Accuracy")
plt.xlabel("Temporal vector size (T)")
plt.show()

plt.title("Accuricies vs T size on unseen data")
plt.plot(average_unseen)
plt.ylabel("Accuracy")
plt.xlabel("Temporal vector size (T)")
plt.show()

In [ ]:
np.save(path+"data_of_t_size",average_a)
print(np.max(average_a,axis=1).shape)

(15,)


In [ ]:
data=np.load(path+"data_of_t_size.npy")
print(data)
for i in range(len(data)):
    plt.title("Accuricies vs T size")
    plt.plot(data[i],label="T="+str(i+1))
plt.legend(loc="lower right")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.show()

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.38057796 0.38239247 0.39254032 ... 0.99616935 0.99543011 0.99475806]
 [0.34301075 0.36196237 0.45248656 ... 0.9938172  0.99455645 0.99462366]
 ...
 [0.33565517 0.37110345 0.49986207 ... 0.99751724 0.99793103 0.99696552]
 [0.36850829 0.35359116 0.42071823 ... 0.99903315 0.99889503 0.99848066]
 [0.39570637 0.38400277 0.34466759 ... 0.99716066 0.99792244 0.99743767]]


In [ ]:
plt.title("Accuricies vs T size on unseen data")
plt.plot(average_unseen)
plt.ylabel("Accuracy")
plt.xlabel("Temporal vector size (T)")
plt.show()

### Dropout rate

In [ ]:
TRIALS=20
TIMES=9
average_a=np.zeros((TIMES,2500))

for i in range(1,TIMES):
    d=dataset()
    x,y=d.generate(STORE=5,y_labels=[[1,0,0],[1,0,0],[0,0,1]])
    SIZE=x.shape[1]
    y=y/10
    n_inputs = x.shape[1]
    m_outputs = len(y[1])
    for j in range(TRIALS):
        X, data_test, Y, labels_test = train_test_split(x, y, test_size=0.20, random_state=42)
        X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)
        
        model = SimpleNeuralNetwork(n_inputs, m_outputs,drop_out_prob=TIMES/10).to(device)
        # Define the loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        l,a=train(model,output=False)
        average_a[i]+=a
    average_a[i]/=TRIALS #get average
    print(i,average_a[-1][-1])
#diplay

plt.title("How dropout rate influences accuracy")
plt.plot(np.max(average_a,axis=1))
plt.ylabel("Accuracy")
plt.xlabel("Dropout rate")
plt.show()

/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
1 0.0


/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
2 0.0


/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


...

flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
6 0.0


/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
7 0.0


/its/home/drs25/miniconda3/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator Ridge from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


flat_detection.avi Length: 338
edge_detection.avi Length: 262
flat_slip_detection.avi Length: 327
8 0.8775978407557353


In [ ]:
plt.title("How dropout rate influences accuracy")

plt.plot([i/10 for i in range(len(np.max(average_a,axis=1)))],np.max(average_a,axis=1))
y_error = np.std(average_a,axis=1)/2
 
plt.errorbar([i/10 for i in range(len(np.max(average_a,axis=1)))],np.max(average_a,axis=1),
             yerr = y_error,
             fmt ='o')
plt.ylabel("Accuracy")
plt.xlabel("Dropout rate")
plt.show()